# A. Imports and initial setup

## A.1 import data

In [1]:
import numpy as np 
import pandas as pd 

from utils import get_stopwords, get_stemmer, tokenizer
dataset_location='../development.json'
# dataset_location='../CorpusTrabalho4.zip'
df = pd.read_json(dataset_location, orient='records')
# print(df.count()) 9998
df.sample(10).head(10)

,description,id,title
5286,Vaga de Emprego: Recepcionista de Mercado\n\n\...,5286,Recepcionista de Mercado
4680,Comprador(a) Cidade: portao Requisitos da vaga...,4680,Oportunidade para Comprador
3427,Atividades Profissionais: Realizar organizacao...,3427,Farmaceutico Corresponsavel
4844,Dirigir veiculos do tipo Carreta;\n\n\nRealiza...,4844,Motorista Carreteiro
8642,Atividades Profissionais: Executar servico ger...,8642,Engenheiro Especialista em Suporte Tecnico
6471,Nosso programa de estagio e diferente. Sabemos...,6471,Vale do Inicio - Programa de Estagio
2839,Sobre a vaga \r\n\r\n\r\n Descricao\r\n\r\n...,2839,Vaga de Caixa De Loja
8424,"Contatar, visitar e entrevistar clientes, com ...",8424,Consultor de Vendas Externas em Sorriso/MT
8684,Atividades Profissionais: Desenvolvimento de s...,8684,Programador .NET Senior
6746,"area de atuacao:comercio varejista, atacadista...",6746,Vaga para Auxiliar de Caixa


## A.2 Stopwords

In [2]:
stopwords= list(get_stopwords())
ind = np.random.randint(0,len(stopwords), size=5)
print('total stop words are %d' % (len(stopwords)))
print([stopwords[i] for i in ind])      

total stop words are 203
['essas', 'teus', 'eram', 'houveriam', 'houvéramos']


## A.3 Stemmer

In [3]:
stemmer= get_stemmer() 
words_test= ['Programador', 'Programa', 'pegaram', 'correram']
print('Before stemming:')
print(words_test)
print('After stemming:')
print([stemmer.stem(w) for w in words_test])

Before stemming:
['Programador', 'Programa', 'pegaram', 'correram']
After stemming:
['program', 'program', 'peg', 'corr']


## A.4 Tokenize the title

In [4]:
tokenfy = lambda x : tokenizer(x, stopwords= stopwords, stemmer=stemmer)
df['token_title'] = df['title'].transform(tokenfy)
df.sample(10).head(10)

,description,id,title,token_title
9028,Atividades Profissionais: Atuar com gestao de ...,9028,Encarregado de setor,encarreg setor
919,Atividades Profissionais: Atendimento ao clie...,919,Designer de Sobrancelha,design sobrancelh
6949,Analista desenvolvedor java liberdade \n\n\nRe...,6949,Vaga para Desenvolvedor JAVA,vag desenvolvedor jav
9749,Analista De Controle De Qualidade Sr em Campin...,9749,Oferta de emprego - Analista De Controle De Qu...,ofert empreg anal control qualidad sr
4648,Requisitos necessarios:Nivel medio concluido o...,4648,Vaga para Auxiliar de Cobranca,vag auxili cobranc
527,Descricao\n\nGeneral Description\nJohnson &amp...,527,Scrum Master Supply Chain - Fixed-term 12 months,scrum mast supply chain fixedterm 12 months
566,Limpeza e esterilizacao e empacotamento de mat...,566,Tecnico de Enfermagem,tecnic enfermag
4609,Atividades Profissionais: Atuar com atendiment...,4609,Representante Comercial Cosmeticos,represent comercial cosmet
2047,"Vendas na area de food service, produto inovad...",2047,Oportunidade para Vendedor,oportun vendedor
3970,Auxiliar de Marceneiro Local: Brasilia / DF,3970,Auxiliar Marceneiro,auxili marceneir


# Titles

## 1. Distinct

In [5]:
distinct_titles= set(df['title'])
distinct_token_titles= set(df['token_title'])
print('distinct titles %d' % (len(distinct_titles)))
print('distinct token titles %d' % (len(distinct_token_titles)))

distinct titles 3804
distinct token titles 3276


## 2. Frequency

In [16]:
dfpivot= df.pivot_table(index='token_title',values='title', aggfunc=len)
print('top 20 job postings')
dfpivot.sort_values('title',ascending=False).head(20)

top 20 job postings


,title
token_title,
vag vendedor,171
vag estagiari,137
vendedor,96
vag consultor vend,76
vag auxili administr,66
vendedor extern,65
vag atendent,64
vag vendedor extern,63
vag auxili servic ger,57
